In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.simplefilter("ignore")

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn import set_config
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
import xgboost as xgboost
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tpot import TPOTClassifier
from sklearn.ensemble import StackingClassifier


In [3]:
df=pd.read_csv('diabetes.csv')

In [4]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness']=np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])




In [6]:
X=df.drop(columns=['Outcome'])
y=df['Outcome']

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)

In [24]:
clf1=RandomForestClassifier(n_estimators=10,random_state=42)
clf2=DecisionTreeClassifier()
clf3=KNeighborsClassifier(n_neighbors=5)

In [8]:
estimator=[
    
    ("Rf",RandomForestClassifier(n_estimators=10,random_state=42)),
    ('Dt',DecisionTreeClassifier()),
    ('Knn',KNeighborsClassifier(n_neighbors=5))
]

In [9]:
clf=StackingClassifier(estimators=estimator,
    final_estimator=LogisticRegression(),
    
    cv=5,
    stack_method='auto',
    n_jobs=-1,
    passthrough=False,
    verbose=0,
)

In [10]:
clf.fit(X_train,y_train)

StackingClassifier(cv=5,
                   estimators=[('Rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('Dt', DecisionTreeClassifier()),
                               ('Knn', KNeighborsClassifier())],
                   final_estimator=LogisticRegression(), n_jobs=-1)

In [11]:
y_pred=clf.predict(X_test)

In [12]:
accuracy_score=accuracy_score(y_test,y_pred)

In [13]:
accuracy_score

0.7760416666666666

In [30]:
cv=RepeatedStratifiedKFold( n_splits=10, n_repeats=5)
cross_val_score(clf,X_train,y_train,cv=cv,scoring='accuracy').mean()

y_pred=clf.predict(X_test)

from sklearn.metrics import accuracy_score

print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[112  14]
 [ 29  37]]
0.7760416666666666
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       126
           1       0.73      0.56      0.63        66

    accuracy                           0.78       192
   macro avg       0.76      0.72      0.74       192
weighted avg       0.77      0.78      0.77       192



In [37]:
print('3-fold cross validation')
for estimators,lable in zip([clf1,clf2,clf3,clf],
                           ['RF',
                           'DT',
                           'KNN',
                           'Stacking classifer']):
    #score=cross_val_score(estimators,X,y,cv=cv,scoring='accuracy')
    score=cross_val_score(estimators,X_train,y_train,cv=cv,scoring='f1_macro')
    #score=cross_val_score(estimators,X_test,y_test,cv=cv,scoring='f1_macro')
    
    print('F1 score:%0.2f (+/-%0.2f)[%s]'
         %(score.mean(),score.std(),lable))

3-fold cross validation
F1 score:0.68 (+/-0.08)[RF]
F1 score:0.66 (+/-0.06)[DT]
F1 score:0.66 (+/-0.07)[KNN]
F1 score:0.69 (+/-0.07)[Stacking classifer]
